In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import gensim
from nltk.corpus import brown
import random
from sklearn.model_selection import KFold
import lightgbm as lgb
import gc
from keras.callbacks.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import keras
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Flatten, LSTM, Dropout, Input, Conv1D, MaxPooling1D
from keras.layers.merge import concatenate
from keras.callbacks.callbacks import EarlyStopping
from keras import backend
from keras.preprocessing.sequence import pad_sequences
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn
import tqdm
from sklearn.model_selection import StratifiedKFold




Using TensorFlow backend.


In [2]:
train = pd.read_csv("../input/google-quest-challenge/train.csv")
test = pd.read_csv("../input/google-quest-challenge/test.csv")

In [3]:
sample_sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")

In [4]:
train.shape

(6079, 41)

In [5]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [6]:
train_labels = train[target_cols].values

In [7]:
len(train.columns)

41

In [8]:
len(target_cols)

30

# fearure engineering

In [9]:
def simple_prepro(s):
    return [w for w in s.replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""]

In [10]:
def simple_prepro_tfidf(s):
    return " ".join([w for w in s.lower().replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""])

In [11]:
qt_max = max([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = max([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = max([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("max lenght of question_title is",qt_max)
print("max lenght of question_body is",qb_max)
print("max lenght of question_answer is",an_max)

max lenght of question_title is 35
max lenght of question_body is 3404
max lenght of question_answer is 8179


In [12]:
qt_max = np.mean([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = np.mean([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = np.mean([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("mean lenght of question_title is",qt_max)
print("mean lenght of question_body is",qb_max)
print("mean lenght of question_answer is",an_max)

mean lenght of question_title is 9.991939463727586
mean lenght of question_body is 161.18703734166803
mean lenght of question_answer is 163.19608488238197


In [13]:
max_word = 35000
max_len = 1024
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_word)

In [51]:
train_drop = train[["question_title", "question_body","answer"]]
test_drop = test[["question_title", "question_body","answer"]]

In [54]:
test_drop

,question_title,question_body,answer
0,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,There is no consequence for leaving corpses an...
1,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,I think it is possible with custom fields.\n\n...
2,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",You do not have armour in the screenshots. Thi...
3,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,Your Western Digital hard drive is disappearin...
4,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,I called two persons who work for Saudia (tick...
...,...,...,...
471,change gb4e enumeration in only one chapter,I am using gb4e for glossing examples in my th...,You can simply reset the exx counter just afte...
472,All anagrams in a File,Source : Microsoft Interview Question\n\nWe ar...,Slightly different approach from the one above...
473,SQL Server: Time Series Table Design,I am creating a time series database that foll...,A foreign key can reference a unique constrain...
474,Is a countered flashback spell exiled?,If I flashback Past in Flames and it is counte...,If you counter a spell cast for its Flashback ...


In [57]:
data = train_drop.append(test_drop)

In [62]:
data.head()

,question_title,question_body,answer
0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin..."
1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...
2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...
3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between..."
4,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...


In [63]:
for i in ["question_title", "question_body", "answer"]:
    tokenizer.fit_on_texts(data[i])    
print(len(tokenizer.word_index))

58158


In [65]:
sequences_title = tokenizer.texts_to_sequences(train["question_title"])
sequences_body = tokenizer.texts_to_sequences(train["question_body"])
sequences_answer = tokenizer.texts_to_sequences(train["answer"])
test_title = tokenizer.texts_to_sequences(test["question_title"])
test_body = tokenizer.texts_to_sequences(test["question_body"])
test_answer = tokenizer.texts_to_sequences(test["answer"])


In [66]:
title_pad = pad_sequences(sequences_title, maxlen=max_len)
body_pad =  pad_sequences(sequences_body, maxlen=max_len)
answer_pad =  pad_sequences(sequences_answer, maxlen=max_len)
tt_title_pad = pad_sequences(test_title, maxlen=max_len)
tt_body_pad =  pad_sequences(test_body, maxlen=max_len)
tt_answer_pad =  pad_sequences(test_answer, maxlen=max_len)

In [67]:
body_pad.shape

(6079, 1024)

In [68]:
train["category"].value_counts()

TECHNOLOGY       2441
STACKOVERFLOW    1253
CULTURE           963
SCIENCE           713
LIFE_ARTS         709
Name: category, dtype: int64

In [69]:
type2int = {type:i for i,type in enumerate(list(set(train["category"])))}
cate = np.identity(5)[np.array(train["category"].apply(lambda x:type2int[x]))].astype(np.float64)
cate_test = np.identity(5)[np.array(test["category"].apply(lambda x:type2int[x]))].astype(np.float64)

In [70]:
cate.shape

(6079, 5)

In [71]:
catego = np.zeros(title_pad.shape)
catego[:cate.shape[0],:cate.shape[1]] = cate

In [72]:
backend.clear_session()

In [73]:
def define_model(length, vocab_size):
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 300)(inputs1)
	conv1 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 300)(inputs2)
	conv2 = Conv1D(filters=128, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 300)(inputs3)
	conv3 = Conv1D(filters=128, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(64, activation='relu')(merged)
	outputs = Dense(len(target_cols), activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
        model.compile(loss='binary_crossentropy', optimizer='adam')
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [74]:
model = define_model(1024, len(tokenizer.word_index))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1024, 300)    17447400    input_1[0][0]                    
____________________________________________________________________________________________

In [25]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

hist = model.fit([title_pad,body_pad,answer_pad], np.array(train_labels), epochs = 50, validation_split=0.2, callbacks = [es])

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4863 samples, validate on 1216 samples
Epoch 1/300
4863/4863 [==============================] - 12s 3ms/step - loss: 0.4333 - accuracy: 0.5533 - val_loss: 0.4177 - val_accuracy: 0.5650
Epoch 2/300
4863/4863 [==============================] - 10s 2ms/step - loss: 0.3888 - accuracy: 0.5732 - val_loss: 0.4012 - val_accuracy: 0.5674
Epoch 3/300
4863/4863 [==============================] - 10s 2ms/step - loss: 0.3633 - accuracy: 0.5844 - val_loss: 0.4011 - val_accuracy: 0.5691
Epoch 4/300
4863/4863 [==============================] - 10s 2ms/step - loss: 0.3388 - accuracy: 0.5938 - val_loss: 0.4001 - val_accuracy: 0.5687
Epoch 5/300
4863/4863 [==============================] - 10s 2ms/step - loss: 0.3195 - accuracy: 0.6007 - val_loss: 0.4041 - val_accuracy: 0.5698
Epoch 6/300
4863/4863 [==============================] - 10s 2ms/step - loss: 0.3065 - accuracy: 0.6043 - val_loss: 0.4063 - val_accuracy: 0.5687
Epoch 7/300
4863/4863 [==============================] - 10s 2ms/step - loss

In [28]:
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [78]:
mission = pd.DataFrame(model.predict([tt_title_pad, tt_body_pad, tt_answer_pad]), columns=target_cols)

In [26]:
sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
sub = pd.merge(sub["qa_id"], mission, left_index=True, right_index= True)
sub.to_csv("submission.csv", index = False)

NameError: name 'test_preds' is not defined

The contribution of the score was not great, but if you use bert etc. instead of brown, I think the score will go up more.